# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Original State Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_2Cases_H2_SmallSteps/')
FigDir       = None #os.path.join(WORKSPACE_PATH, '../Desktop/Paper_Figures_DeepONet/')


tStratch     = 1.
SOLVER       = 'BDF'

valid_perc   = 20.

FixedMinVal  = 1.e-14


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

NProcs       = 1

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass

## Reading Data

In [4]:
j=0
for i in range(NProcs):
    Data0_             = pd.read_csv(OutputDir+'/Orig/train_file/ext/y0.csv.'+str(i+1),   header=None, skiprows=1)
    Data_              = pd.read_csv(OutputDir+'/Orig/train_file/ext/yEnd.csv.'+str(i+1), header=None, skiprows=1)
    if (j==0):
        Columns0       = list(pd.read_csv(OutputDir+'/Orig/train_file/ext/y0.csv.'+str(i+1), header=0).columns)
        Columns0[0]    = 't0'
        Columns        = list(pd.read_csv(OutputDir+'/Orig/train_file/ext/yEnd.csv.'+str(i+1),  header=0).columns)
    Data0_.columns         = Columns0
    Data_.columns          = Columns
    if (j==0):
        Data0          = Data0_.copy()
        DataEnd        = Data_.copy()
    else:
        Data0          = pd.concat([Data0, Data0_],  axis=0, ignore_index=True)
        DataEnd        = pd.concat([DataEnd, Data_], axis=0, ignore_index=True)
    j+=1
    
Data  = pd.concat([Data0, DataEnd], axis=1)

In [5]:
# iNeg = []
# for i in range(len(Data)):
#     if any(Data.iloc[i] < 0):
#         iNeg.append(i)

# Data  = Data.clip(1.e-30, 1.e10)
# Data  = Data.drop(iNeg)

In [6]:
Data[Columns0[1::]] = np.log10(Data[Columns0[1::]]) #+ 30.
Data[Columns[1::]]  = np.log10(Data[Columns[1::]])  #+ 30.

In [7]:
Data        = Data.dropna(axis=0)

In [8]:
InputVar    = Columns0
InputVar[0] = 't'
Vars        = Columns

In [9]:
XX = Data.to_numpy()

In [10]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=30, random_state=0).fit(XX)
# kmeans.labels_

array([22, 22,  0, ...,  8,  8,  8], dtype=int32)

In [ ]:
# plt.scatter(kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,10])

In [ ]:
# plt.scatter(Data['T0'][kmeans.labels_==0], Data['N'][kmeans.labels_==0])
# plt.scatter(Data['T0'][kmeans.labels_==1], Data['N'][kmeans.labels_==1])

## Splitting Data and Writing Database 

In [11]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/Orig/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/')
except:
    pass

In [12]:
data_id    = 'pts'

DataNoZero           = Data #[kmeans.labels_ == 0]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)